In [35]:
import gspread
import time 
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd


In [125]:
class ScrapearGMaps:


    def __init__(self) -> None:
        # Iniciamos el webdriver
        self.driver = webdriver.Chrome()
        

    # Funcion para scrollear
    def scroll_page (self):
        # Identificamos el objeto hacia el que queremos hacer el scroll
        section_loading = self.driver.find_element(By.CLASS_NAME, 'TFQHme ')

        # Hacer scroll en los resultados
        for _ in range(30):  # Ajustar el rango según la cantidad de scrolls que necesites
            scrollable_div = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
            self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(3)  # Esperar un poco antes de hacer el siguiente scroll

    # Funcion para obtener el nombre
    def get_name (self):
        try:
            # Por medio del Xpath extraemos el texto del nombre
            return self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text
        except:
            # Retornamos SD de sin datos en el caso de que la extraccion sea fallida
            return "SD"
        
    # Funcion para extraer las coordenadas      
    def get_coord (self,link):
        # Usamos expresion regular para extraer las coordenadas desde el link 
        coordenadas = re.search(r"!3d-?\d\d?\.\d{4,8}!4d-?\d\d?\.\d{4,8}", link).group()
        # Separamos para que queden unicamente los datos numericos y separados
        coordenada = coordenadas.replace('!3d','')
        # Retornamos una tupla con los dos datos
        return tuple(coordenada.split('!4d'))
    
    # Funcion para obtener el Gmap_id
    def get_gmapid(self,link):
        # Guardamos como variable el patron encontrado para hallar el Gmap_id
        patron = r'0x[0-9a-fA-F]+:0x[0-9a-fA-F]+'

        try:
            # Usando expresion regular extraemos el Gmap_id usando el patron encontrado
            match = re.search(patron, link)
            # Agrupamos los datos y los retornamos
            extracted_data = match.group()
            return extracted_data

        except:
            # En caso de no ser excitosa la extraccion retornara un mensaje de extraccion fallida
            return 'Extraccion fallida'
        
    # Funcion para hallar la categoria del restaurante
    def get_category (self):
        # Por medio del Xpath se busca la parte que contenga texto y se guarda en una variable
        categoria=self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/span[1]/span/button').text
        # Se retorna la categoria extraida
        return categoria
    
    # Funcion para obtener el Rating del lugar
    def get_rating (self):
        # Usamos el Xpath para hallar el texto dentro de la ubicacion
        rating = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]').text
        # Reemplazamos las comas por puntos para poder transformalo en formato Float
        rating = rating.replace(',','.')
        # Retornamos el valor tipo float
        return float(rating)
    
    # Funcion para hallar la cantidad de reviews por sitio
    def get_reviews (self):
        # Usando el Xpath extraemos la cantidad de reviews que tiene cada sitio
        reviews = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span').text
        # Usando Replace eliminamos algunos elementos para poder convertirlo en int
        reviews = reviews.replace('.','')
        reviews = reviews.replace('(','')
        reviews = reviews.replace(')','')
        # Retornamos el valor numerico
        return int(reviews)
    
    # Funcion para hallar la direccion del lugar
    def get_direccion (self):
            try:
                # Intenta extraer la direccion por medio de la ruta dada y retornarla en tipo texto
                direccion = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]').text

                return direccion
            except:
                # Genera una excepcion con la otra posible ruta de la direccion y la retorna en tipo texto
                direccion = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[11]/div[3]/button/div/div[2]/div[1]').text

                return direccion

    # Creamos una funcion para extraer la ciudad desde la direccion
    def extraer_ciudad(self, direccion):

        try:
            # Dividir la dirección en partes usando la coma como separador
            partes = direccion.split(", ")

            # Asegurarse de que haya al menos tres partes (ciudad, estado_código postal)
            if len(partes) >= 3:
                ciudad = partes[1]  # Extraer la 3ra parte como ciudad
                return ciudad
            else:
                return None # Devolver None para ciudad y estado si tiene la configuración planteada
        except IndexError:
            return None  # Devolver None para ciudad y estado si la extracción falla
        
    # Funcion para extraer el Id de usuario
    def extraer_id_usuario (self,link):

        try:
            # Usar una expresión regular para encontrar la parte numérica
            match = re.search(r'/(\d+)/reviews', link)

            # Extraer y mostrar la parte numérica si se encuentra
            numeric_part = match.group(1)
            return int(numeric_part)
        except:
            return None              
    
    # Funcion que realiza el scraping
    def scraping (self,link):

        sitios = pd.read_parquet('../../Data/Parquet/Sitios_nuevo.parquet')
        busquedas = ['Restaurantes mexicanos, Pinellas Park, EEUU']
        
        # Se crean las listas para almacenar los datos del dataframe sitios
        name = []
        gmap_id = []
        latitud = []
        longitud = []
        categoria = []
        avg_rating = []
        reviews = []
        ciudad = []

        # Se crean las listas que  van a almacenar los datos del dataframe reviews
        reviewid = []
        userid = []
        username = []
        rating = []
        text = []
        gmap_idreseña = []
        businessname = []
        categoriareview = []
        ciudadreview = []

        # Usando el driver se accede al link
        self.driver.get(link)
        # Se da una espera de 5 segundos para que cargue la pagina
        time.sleep(5)
        buscador = self.driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
        for i in range (0,len(busquedas)):
            buscador.clear()
            buscador.click()
            buscador.send_keys(busquedas[i])

            buscar = self.driver.find_element(By.XPATH, '//*[@id="searchbox-searchbutton"]')
            buscar.click()
            time.sleep(2)
            # Se llama la funcion que hace scroll 
            self.scroll_page()
            # Se guarda en una variable las rutas de todos los lugares encontrados
            lugares = self.driver.find_elements(By.CLASS_NAME, 'hfpxzc')
            cantidad = len(lugares)
            # Por medio de un ciclo se itera la lista anteriormente creada permitiendo entrar sitio por sitio para extraer la informacion
            for i in range(0,cantidad):
                
                try:
                    # Se ubica en el lugar segun el ciclo
                    lugar = lugares[i]
                    # Se escrolea hasta el sitio
                    scrollable_div = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
                    self.driver.execute_script("arguments[0].scrollIntoView();", lugar)
                    # Se esperan dos segundos y se da click para abrir la informacion del lugar
                    time.sleep(2)
                    lugar.click()
                    # Se esperan 4 segundos para que la informacion cargue correctamente
                    time.sleep(4)
                    mapid_temp = self.get_gmapid(self.driver.current_url)
                    if mapid_temp not in sitios['gmap_id'].values :
                        # Se llaman todas las funciones que extraen la informacion necesaria y se guardan en variables temporales
                        name_temp = self.get_name()
                        
                        coordenadas = self.get_coord(self.driver.current_url)
                        categoria_temp = self.get_category()
                        rating_temp = self.get_rating()
                        reviews_temp = self.get_reviews()
                        direccion = self.get_direccion()
                        ciudad_temp = self.extraer_ciudad(direccion)
                    

                        name.append(name_temp)
                        gmap_id.append(mapid_temp)
                        latitud.append(coordenadas[0])
                        longitud.append(coordenadas[1])
                        categoria.append(categoria_temp)
                        avg_rating.append(rating_temp)
                        reviews.append(reviews_temp)
                        ciudad.append(ciudad_temp)
                        time.sleep(2)

                        try:
                            # Se busca el apartado de reseñas y se da click
                            opiniones = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]/div[2]/div[2]')
                            opiniones.click()
                            # Se esperan 2 segundos y se busca el boton para ordenarlos por mas recientes
                            time.sleep(4)
                            ordenar = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]/div[7]/div[2]/button/span')
                            ordenar.click()
                            time.sleep(4)
                            # Despues de esperar 2 segundos se busca el boton de ordenar por mas recientes y se le da click
                            orden = self.driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]')
                            orden.click()
                            # Se esperan 2 segundos y se escrollea un poco para extraer las reviews mas recientes
                            time.sleep(2)
                            scrollable_div = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
                            for i in range (0,2):
                                self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
                                time.sleep(3)
                            # Se guarda como variable las rutas de las reseñas encontradas
                            reseñas = self.driver.find_elements(By.CLASS_NAME, 'jJc9Ad ')
                            cantidad_reviews = len(reseñas)

                            for i in range (0,cantidad_reviews):
                                    
                                buton = reseñas[i].find_element(By.CLASS_NAME, 'WEBjve')
                                data_review_id = buton.get_attribute('data-review-id')

                                try:
                                    # Localizar el botón usando su clase (puedes usar cualquier otro selector adecuado)
                                    element = reseñas[i].find_element(By.CLASS_NAME, 'al6Kxe')

                                    # Obtener el valor del atributo data-href
                                    data_href = element.get_attribute('data-href')

                                    # Extraer la parte numérica del enlace usando una expresión regular
                                    match = re.search(r'\d+', data_href)
                                    if match:
                                        userid_temp = match.group(0)  
                                    else:
                                        userid_temp = None
                                except:
                                    userid_temp = None

                                # Intentamos extraer el nombre del usuario
                                try:
                                    nombreuser_temp = reseñas[i].find_element(By.CLASS_NAME, 'd4r55 ').text
                                except:
                                    nombreuser_temp = None
                                
                                # intentamos extraer el rating dado en la reseña
                                try:
                                    # Localizar el elemento que contiene las estrellas
                                    elemento=reseñas[i].find_element(By.CLASS_NAME, 'kvMYJc')
                                    # Localiza el elemento por su clase

                                    # Obtener el valor del atributo aria-label
                                    aria_label = elemento.get_attribute('aria-label')

                                    # Extraer el número de estrellas del aria-label
                                    calificacion_temp = int(aria_label.split()[0])  # Asumiendo que el formato es siempre 'X estrellas'
                                except:
                                    calificacion_temp = None

                                # Tratamos de extraer el texto del review
                                botonmas = self.driver.find_element(By.CLASS_NAME, 'w8nwRe')
                                botonmas.click()

                                try:
                                    texto_temp = reseñas[i].find_element(By.CLASS_NAME, 'wiI7pd').text
                                except:
                                    texto_temp = None
                                
                                reviewid.append(data_review_id)
                                userid.append(userid_temp)
                                username.append(nombreuser_temp)
                                rating.append(calificacion_temp)
                                text.append(texto_temp)
                                gmap_idreseña.append(mapid_temp)
                                businessname.append(name_temp)
                                categoriareview.append(categoria_temp)
                                ciudadreview.append(ciudad_temp)

                        except:

                            reviewid.append(None)
                            userid.append(None)
                            username.append(None)
                            rating.append(None)
                            text.append(None)
                            gmap_idreseña.append(None)
                            businessname.append(None)
                            categoriareview.append(None)
                            ciudadreview.append(None)
                except:

                    name.append(None)
                    gmap_id.append(None)
                    latitud.append(None)
                    longitud.append(None)
                    categoria.append(None)
                    avg_rating.append(None)
                    reviews.append(None)
                    ciudad.append(None)
                    time.sleep(2)
                



        sitios = pd.DataFrame({'name':name,
                            'gmap_id':gmap_id,
                            'latitude':latitud,
                            'longitude':longitud,
                            'category':categoria,
                            'avg_rating':avg_rating,
                            'num_of_reviews':reviews,
                            'city':ciudad})
        
        reviews = pd.DataFrame({'review_id':reviewid,
                            'user_id':userid,
                            'user_name':username,
                            'rating':rating,
                            'text':text,
                            'gmap_id':gmap_idreseña,
                            'business_name':businessname,
                            'category':categoriareview,
                            'city':ciudadreview})
            
        return sitios, reviews

In [126]:
link = 'https://www.google.com/maps/search/Restaurantes,+Florida,+EEUU/@28.4613115,-91.2139021,6z/data=!4m2!2m1!6e5?entry=ttu'

gscraping = ScrapearGMaps()

sitios, reviews = gscraping.scraping(link)

gscraping.driver.quit()

In [134]:
archivo_sitios = pd.read_parquet('../../Data/Parquet/Sitios_nuevo.parquet')

In [136]:
archivo_sitios.dropna(inplace=True)
archivo_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534 entries, 0 to 119
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            534 non-null    object 
 1   gmap_id         534 non-null    object 
 2   latitude        534 non-null    object 
 3   longitude       534 non-null    object 
 4   category        534 non-null    object 
 5   avg_rating      534 non-null    float64
 6   num_of_reviews  534 non-null    float64
 7   city            534 non-null    object 
dtypes: float64(2), object(6)
memory usage: 37.5+ KB


In [128]:
sitios2 = sitios, archivo_sitios

In [129]:
sitios_ultimo = pd.concat(sitios2)
sitios_ultimo

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,city
0,Los Homies Taqueria,0x88c2e5b78c7489ef:0xfb1fa5ca906dc9cd,27.8413044,-82.6991762,Restaurante mexicano,4.5,306.0,Pinellas Park
1,Gloria's,0x88c2e4857ab9dbe5:0x8e6e72a5c03cff9d,27.8366869,-82.700276,Restaurante mexicano,4.4,9.0,FL 33781
2,None,None,None,None,None,NaN,NaN,None
3,Chinelos Taqueria,0x88c2e3069d888353:0xdba7cff8ef64b268,27.820642,-82.7208642,Restaurante mexicano,4.9,467.0,Kenneth City
4,Señor Taco Mexican Restaurant,0x88c2e4b011ad6205:0x3ab742c1342fbb34,27.8395858,-82.7246405,Restaurante,4.4,1141.0,Pinellas Park
...,...,...,...,...,...,...,...,...
115,Tijuana Flats,0x88e77a63807e9acb:0x1c17e8ce394ce0ba,28.5717984,-81.3892391,Taquería,4.3,993.0,Orlando
116,Tijuana Flats,0x88dd872d12905ef5:0xb4ee352468748dfe,28.36666,-81.426033,Taquería,4.5,1375.0,Orlando
117,Hunger Street Tacos,0x88e7707b9b2f8579:0x533ca4e1307ae855,28.5934219,-81.3777168,Restaurante mexicano,4.4,1907.0,Winter Park
118,Moe's Southwest Grill,0x88e77edfac4210c9:0x425b631cc97ecb38,28.4735501,-81.4652353,Restaurante mexicano,3.2,299.0,Orlando


In [137]:
archivo_reviews = pd.read_parquet('../../Data/Parquet/reviews_nuevo.parquet')

In [138]:
archivo_reviews.dropna(subset='review_id', inplace=True)
archivo_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9496 entries, 0 to 2612
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   review_id      9496 non-null   object 
 1   user_id        9496 non-null   object 
 2   user_name      9496 non-null   object 
 3   rating         9496 non-null   float64
 4   text           7081 non-null   object 
 5   gmap_id        9496 non-null   object 
 6   business_name  9496 non-null   object 
 7   category       9496 non-null   object 
 8   city           9496 non-null   object 
dtypes: float64(1), object(8)
memory usage: 741.9+ KB


In [131]:
reviews2 = reviews, archivo_reviews

reviews_ultimo = pd.concat(reviews2)
reviews_ultimo

,review_id,user_id,user_name,rating,text,gmap_id,business_name,category,city
0,ChZDSUhNMG9nS0VJQ0FnSURMX3RMQUtBEAE,116093912001907653593,Manuel Fernando Pirazan Africano,5.0,None,0x88c2e5b78c7489ef:0xfb1fa5ca906dc9cd,Los Homies Taqueria,Restaurante mexicano,Pinellas Park
1,ChdDSUhNMG9nS0VJQ0FnSURMM3ItOXNnRRAB,102693378429711249224,Gabriela Padilla Lopez,5.0,None,0x88c2e5b78c7489ef:0xfb1fa5ca906dc9cd,Los Homies Taqueria,Restaurante mexicano,Pinellas Park
2,ChZDSUhNMG9nS0VJQ0FnSURMbnVYT2F3EAE,115292094493632469797,Caitlin Elizabeth,5.0,"Tacos legítimos, el queso es realmente bueno y...",0x88c2e5b78c7489ef:0xfb1fa5ca906dc9cd,Los Homies Taqueria,Restaurante mexicano,Pinellas Park
3,ChZDSUhNMG9nS0VJQ0FnSUNMbFlXZ0R3EAE,101094124122700404279,joshua saunders,2.0,None,0x88c2e5b78c7489ef:0xfb1fa5ca906dc9cd,Los Homies Taqueria,Restaurante mexicano,Pinellas Park
4,ChZDSUhNMG9nS0VJQ0FnSUN4anRtMlBREAE,107777809100806802304,Jovan Milakovic,5.0,"Un pequeño gran lugar auténtico, la comida es ...",0x88c2e5b78c7489ef:0xfb1fa5ca906dc9cd,Los Homies Taqueria,Restaurante mexicano,Pinellas Park
...,...,...,...,...,...,...,...,...,...
2609,ChdDSUhNMG9nS0VJQ0FnSUMxbzg2d3dRRRAB,108277700942414758865,Melissa Dinh,1.0,Absolutamente la peor experiencia que he tenid...,0x88e77ea9a255bfdf:0xe0d9fde1fd07b4d1,Moe's Southwest Grill,Restaurante mexicano,Orlando
2610,ChZDSUhNMG9nS0VJQ0FnSUMxeHQ2cU53EAE,105559442342327966865,Ethan coan,5.0,Ambiente muy amigable con comida deliciosa. El...,0x88e77ea9a255bfdf:0xe0d9fde1fd07b4d1,Moe's Southwest Grill,Restaurante mexicano,Orlando
2611,ChdDSUhNMG9nS0VJQ0FnSUMxeHQ3c25BRRAB,113357465804118875047,Luis Fernando Bueno Pena,5.0,¡Buena comida! El personal fue muy amable. Def...,0x88e77ea9a255bfdf:0xe0d9fde1fd07b4d1,Moe's Southwest Grill,Restaurante mexicano,Orlando
2612,ChdDSUhNMG9nS0VJQ0FnSUMxX09PLW1RRRAB,100988594044464700465,MrNigh†mare,5.0,None,0x88e77ea9a255bfdf:0xe0d9fde1fd07b4d1,Moe's Southwest Grill,Restaurante mexicano,Orlando


In [104]:
sitios_ultimo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 506 entries, 0 to 119
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            494 non-null    object 
 1   gmap_id         494 non-null    object 
 2   latitude        494 non-null    object 
 3   longitude       494 non-null    object 
 4   category        494 non-null    object 
 5   avg_rating      494 non-null    float64
 6   num_of_reviews  494 non-null    float64
 7   city            494 non-null    object 
dtypes: float64(2), object(6)
memory usage: 35.6+ KB


In [139]:
archivo_sitios.to_parquet('../../Data/Parquet/Sitios_nuevo.parquet')

In [140]:
archivo_reviews.to_parquet('../../Data/Parquet/reviews_nuevo.parquet')

In [35]:
sitios2 = sitios[sitios['category'].str.contains('Mex', case=False, na=False)]

Recomiendo hacer el scraping busqueda por busqueda, ya que al tratarlas de hacer todas a la vez el buscador nos desconectara y se perdera toda la información

In [ ]:
'Restaurantes mexicanos, Orlando, EEUU'
'Restaurantes mexicanos, Miami, EEUU'
'Restaurantes mexicanos, Tampa, EEUU'
'Restaurantes mexicanos, Clearwater, EEUU'
'Restaurantes mexicanos, Saint Petesburg, EEUU'
'Restaurantes mexicanos, Brandon, EEUU'
'Restaurantes mexicanos, Largo, EEUU'
'Restaurantes mexicanos, Palm Harbor, EEUU'
'Restaurantes mexicanos, Dunedin, EEUU'
'Restaurantes mexicanos, Pinellas Park, EEUU'

In [45]:
import time
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://www.google.com/maps/search/Restaurantes,+Florida,+EEUU/@28.4613115,-91.2139021,6z/data=!4m2!2m1!6e5?entry=ttu')
time.sleep(5)

section_loading = driver.find_element(By.CLASS_NAME, 'TFQHme ')

# Hacer scroll en los resultados
for _ in range(30):  # Ajustar el rango según la cantidad de scrolls que necesites
    scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
    time.sleep(2)  # Esperar un poco antes de hacer el siguiente scroll


lugares = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
lugar = lugares[0]

time.sleep(2)
lugar.click()
time.sleep (2)
nombre=driver.find_element(By.CLASS_NAME, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text
print(nombre)

url = driver.current_url

print(url)

InvalidSelectorException: Message: invalid selector
from javascript error: {"status":32,"value":"An invalid or illegal selector was specified"}
  (Session info: chrome=126.0.6478.116); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF70C1B3E32+31618]
	(No symbol) [0x00007FF70C12B099]
	(No symbol) [0x00007FF70BFE888A]
	(No symbol) [0x00007FF70BFEE6BE]
	(No symbol) [0x00007FF70BFF1011]
	(No symbol) [0x00007FF70BFF10B0]
	(No symbol) [0x00007FF70C03805C]
	(No symbol) [0x00007FF70C03862C]
	(No symbol) [0x00007FF70C07F787]
	(No symbol) [0x00007FF70C05D14F]
	(No symbol) [0x00007FF70C07CA80]
	(No symbol) [0x00007FF70C05CEB3]
	(No symbol) [0x00007FF70C02A46B]
	(No symbol) [0x00007FF70C02B001]
	GetHandleVerifier [0x00007FF70C4B9FFD+3202381]
	GetHandleVerifier [0x00007FF70C506A1D+3516269]
	GetHandleVerifier [0x00007FF70C4FC490+3473888]
	GetHandleVerifier [0x000

In [46]:
opiniones = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]/div[2]/div[2]')
opiniones.click()

time.sleep(2)
ordenar = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]/div[7]/div[2]/button/span')
ordenar.click()
time.sleep(2)
orden = driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]')

orden.click()

scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
for i in range (0,2):
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
    time.sleep(3)

Nombres = driver.find_elements(By.CLASS_NAME, 'd4r55 ')

In [47]:
reseñas = driver.find_elements(By.CLASS_NAME, 'jJc9Ad ')
cantidad_reviews = len(reseñas)
cantidad = len(Nombres)
Nombres

[<selenium.webdriver.remote.webelement.WebElement (session="46261430fcd7d50367ed0a04d63d0ac2", element="f.0F7C49085E0DF1A923335040CCB3FA40.d.254EADAE797820BC20D1CCDC36A87642.e.279")>,
 <selenium.webdriver.remote.webelement.WebElement (session="46261430fcd7d50367ed0a04d63d0ac2", element="f.0F7C49085E0DF1A923335040CCB3FA40.d.254EADAE797820BC20D1CCDC36A87642.e.280")>,
 <selenium.webdriver.remote.webelement.WebElement (session="46261430fcd7d50367ed0a04d63d0ac2", element="f.0F7C49085E0DF1A923335040CCB3FA40.d.254EADAE797820BC20D1CCDC36A87642.e.281")>,
 <selenium.webdriver.remote.webelement.WebElement (session="46261430fcd7d50367ed0a04d63d0ac2", element="f.0F7C49085E0DF1A923335040CCB3FA40.d.254EADAE797820BC20D1CCDC36A87642.e.282")>,
 <selenium.webdriver.remote.webelement.WebElement (session="46261430fcd7d50367ed0a04d63d0ac2", element="f.0F7C49085E0DF1A923335040CCB3FA40.d.254EADAE797820BC20D1CCDC36A87642.e.283")>,
 <selenium.webdriver.remote.webelement.WebElement (session="46261430fcd7d50367ed

In [50]:
boton = reseñas[0].find_element(By.CLASS_NAME, 'WEBjve')
data_review_id = boton.get_attribute('data-review-id')

print(data_review_id)

ChdDSUhNMG9nS0VJQ0FnSURMbXFqU2tBRRAB


In [88]:
reseñas[0].find_element(By.CLASS_NAME,'d4r55 ').text

'juan freitas'

In [91]:
# Localizar el botón usando su clase (puedes usar cualquier otro selector adecuado)
id1 = reseñas[0].find_element(By.CLASS_NAME, 'al6Kxe')

# Obtener el valor del atributo data-href
data_href = id1.get_attribute('data-href')

# Extraer la parte numérica del enlace usando una expresión regular
match = re.search(r'\d+', data_href)
if match:
    numero = match.group(0)
    print(numero)  # Debería imprimir '111121299400135924938'


114018159987733753653


In [80]:
# Localizar el elemento que contiene las estrellas
elemento=reseñas[0].find_element(By.CLASS_NAME, 'kvMYJc')
  # Localiza el elemento por su clase

# Obtener el valor del atributo aria-label
aria_label = elemento.get_attribute('aria-label')

# Extraer el número de estrellas del aria-label
calificacion = int(aria_label.split()[0])  # Asumiendo que el formato es siempre 'X estrellas'

print(calificacion)  # Debería imprimir 1 en este caso

5


In [81]:
botonmas = driver.find_element(By.CLASS_NAME, 'w8nwRe')
botonmas.click()

In [84]:
texto = reseñas[0].find_element(By.CLASS_NAME, 'wiI7pd').text
texto

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=126.0.6478.116); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF70C1B3E32+31618]
	(No symbol) [0x00007FF70C12B099]
	(No symbol) [0x00007FF70BFE888A]
	(No symbol) [0x00007FF70C038524]
	(No symbol) [0x00007FF70C03862C]
	(No symbol) [0x00007FF70C02BCDC]
	(No symbol) [0x00007FF70C05D14F]
	(No symbol) [0x00007FF70C02BBA6]
	(No symbol) [0x00007FF70C05D320]
	(No symbol) [0x00007FF70C07CA80]
	(No symbol) [0x00007FF70C05CEB3]
	(No symbol) [0x00007FF70C02A46B]
	(No symbol) [0x00007FF70C02B001]
	GetHandleVerifier [0x00007FF70C4B9FFD+3202381]
	GetHandleVerifier [0x00007FF70C506A1D+3516269]
	GetHandleVerifier [0x00007FF70C4FC490+3473888]
	GetHandleVerifier [0x00007FF70C265D36+760454]
	(No symbol) [0x00007FF70C136B3F]


In [20]:
buttons = driver.find_elements(By.CLASS_NAME, 'al6Kxe')

# Extrae y almacena los enlaces
links = [button.get_attribute('data-href') for button in buttons]

In [22]:
link = links[0]

# Usar una expresión regular para encontrar la parte numérica
match = re.search(r'/(\d+)/reviews', link)

# Extraer y mostrar la parte numérica si se encuentra
if match:
    numeric_part = match.group(1)
    print(numeric_part)
else:
    print("No se encontró una parte numérica en el enlace.")

108247853503606105731


In [59]:
nombre=driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text

print(nombre)

El Cilantrillo Restaurant


In [149]:
nombre

'Estefan Kitchen Orlando'

In [11]:
url = driver.current_url

In [13]:
print(url)

https://www.google.com/maps/place/El+Cilantrillo+Restaurant/@28.4613115,-91.2139021,6z/data=!4m11!1m3!2m2!1sRestaurantes,+Florida,+EEUU!6e5!3m6!1s0x88e77dddfa7e805d:0xf115633fcf003d5!8m2!3d28.4501349!4d-81.3969286!15sChtSZXN0YXVyYW50ZXMsIEZsb3JpZGEsIEVFVVVaGyIZcmVzdGF1cmFudGVzIGZsb3JpZGEgZWV1dZIBF3B1ZXJ0b19yaWNhbl9yZXN0YXVyYW50mgEkQ2hkRFNVaE5NRzluUzBWSlEwRm5TVVJoY0V0TWNHcFJSUkFC4AEA!16s%2Fg%2F11mvn2dhtn?entry=ttu


In [151]:
coordenadas = re.search(r"!3d-?\d\d?\.\d{4,8}!4d-?\d\d?\.\d{4,8}", url).group()
coordenada = coordenadas.replace('!3d','')

final = tuple(coordenada.split('!4d'))

final

('28.3455145', '-81.6086358')

In [152]:

pattern = r'0x[0-9a-fA-F]+:0x[0-9a-fA-F]+'

match = re.search(pattern, url)

if match:
    extracted_data = match.group()
    print("Dato extraído:", extracted_data)
else:
    print("No se encontró el patrón.")
# 0x88e77dddfa7e805d:0xf115633fcf003d5

Dato extraído: 0x88dd7dc69b3a06cb:0xfa20cb8563a46242


In [153]:

categoria=driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/span[1]/span/button').text

print(categoria)

Restaurante


In [154]:
rating = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]').text
rating = rating.replace(',','.')
float(rating)

4.1

In [155]:
reviews = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span').text
reviews = reviews.replace('.','')
reviews = reviews.replace('(','')
reviews = reviews.replace(')','')
int(reviews)

1385

In [157]:
direccion = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]').text
# //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]
# //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[11]/div[3]/button/div/div[2]/div[1]
direccion

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]"}
  (Session info: chrome=126.0.6478.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF633833E32+31618]
	(No symbol) [0x00007FF6337AB099]
	(No symbol) [0x00007FF63366888A]
	(No symbol) [0x00007FF6336B8524]
	(No symbol) [0x00007FF6336B862C]
	(No symbol) [0x00007FF6336FF787]
	(No symbol) [0x00007FF6336DD14F]
	(No symbol) [0x00007FF6336FCA80]
	(No symbol) [0x00007FF6336DCEB3]
	(No symbol) [0x00007FF6336AA46B]
	(No symbol) [0x00007FF6336AB001]
	GetHandleVerifier [0x00007FF633B39FFD+3202381]
	GetHandleVerifier [0x00007FF633B86A1D+3516269]
	GetHandleVerifier [0x00007FF633B7C490+3473888]
	GetHandleVerifier [0x00007FF6338E5D36+760454]
	(No symbol) 

In [99]:
lista = [1,2,3]
lista2 = [4,5,6]
len(lista)

3

In [103]:
import pandas as pd

df = pd.DataFrame({'a':lista, 'b':lista2})

df

,a,b
0,1,4
1,2,5
2,3,6


In [66]:
lugar = lugares[21]

time.sleep(2)
lugar.click()

In [65]:
scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
driver.execute_script("arguments[0].scrollIntoView();", lugar)